<div style="text-align: right">Maxwell You</div>

# A3: A\*, IDS, and Effective Branching Factor

For this assignment, I have implemented the Recursive Best-First Search
implementation of the A\* algorithm given in class. Also in this notebook, I have included my `iterativeDeepeningSearch` functions so that I can compare their efficiecies at the end. To aid in this comparison, I have also written an Effective Branching Factor function `ebf` that returns an estimate of the effective
branching factor for a search algorithm applied to a search problem.

The main functions of interest in this notebook are:

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`,
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`,
   - `ebf(nNodes, depth, precision=0.01)`, returns number of nodes expanded and depth reached during a search

`iterativeDeepeningSearch` and `aStarSearch` are applied to several eight-tile sliding puzzle
problems. The main functions necessary to carry out `iterativeDeepeningSearch` are:

  * `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. Each action includes a step cost of 1. For example, if all four actions are possible from this state, return [('left', 1), ('right', 1), ('up', 1), ('down', 1)].
  * `takeActionF_8p(state, action)`: return the state that results from applying `action` in `state` and the cost of the one step
  * `goalTestF_8p(state, goal)`: return true if state equals goal state
  
To compare both searches, I have provided a table with solution path depth, number of nodes generated, and the effective branching factor, and discuss the results. The table is produced by the following function:

   * `runExperiment(goalState1, goalState2, goalState3, heuristics)`: returns a table comparing different aspects of IDS and A\* search. `heuristics` is a list of heuristics to test

The searches below will be using a global variable `num_nodes` in order to keep track of the number of nodes generated during their execution.

In [1]:
global num_nodes
num_nodes = 0

## 8-Puzzle Functions

### Finding tiles

A generic function to find the positions of any tile on the board. Finding the blank is just a special case of this function (i.e. `findTile_8p(state, 0)`. We want to find the blank in order to determine the possible moves we can make.

In [2]:
def findTile_8p(state, tile):
    """
    Returns tuple with coordinates of tile
    Assumes that there will be a blank (0) in the input list
    :param state:
    :return (row, col):
    """
    row = -1
    col = -1
    for i in range(len(state)):
        if i < 3 and state[i] == tile:
                row = 0                # in 1st row if (0 <= index in list < 3)
                col = i                # col is the offset within first 3 indices
                break
        elif i < 6 and state[i] == tile:
                row = 1                # in 2nd row if (3 <= index in list < 6)
                col = 2 - (5 - i)      # col is the offset within 3rd-5th indices
                break
        elif i < 9 and state[i] == tile:
                row = 2                # in 3rd row if (6 <= index in list < 9)
                col = 2 - (8 - i)      # col is the offset within 6th-8th indices
                break
    return (row, col)

### Finding actions

In [3]:
def actionsF_8p(state):
    """
    Returns list of valid actions based on the position of the blank
    Assumes a valid state with blank is provided
    :param state:
    :return actions:
    """
    actions = []
    row, col = findTile_8p(state, 0)
    if 0 <= col - 1 < 3:
        actions.append(('left', 1))
    if 0 <= col + 1 < 3:
        actions.append(('right', 1))
    if 0 <= row - 1 < 3:
        actions.append(('up', 1))
    if 0 <= row + 1 < 3:
        actions.append(('down', 1))

    return actions

### Translating grid coordinates to list index

takeActionF_8p will use this to switch the tiles around the game board.

In [4]:
def tileIndexInList(row, col):
    """
    Returns index of blank in the list based on its row, col in the gameboard
    Assumes valid row, col are given
    :param row:
    :param col:
    :return index:
    """
    index = -1
    if row == 0:
        index = col
    if row == 1:
        index = 3 + col
    if row == 2:
        index = 6 + col
    return index

### Taking actions

Manipulating the list indices to change the state of the game board.

In [5]:
def takeActionF_8p(state, action):
    """
    Returns the modified state list by applying the action
    Assumes only valid actions are given
    :param state:
    :param action:
    :return state:
    """
    stateCopy = state[:]  # copy state into a new list, so we dont modify the original
    row, col = findTile_8p(state, 0)
    if action[0] == 'up':  # get direction from action tuple
        moveFrom = tileIndexInList(row, col)  # index of blank in the list
        stateCopy[moveFrom] = stateCopy[moveFrom - 3]  # swap blank with tile above
        stateCopy[moveFrom - 3] = 0  # place blank in tile above

    elif action[0] == 'down':  # get direction from action tuple
        moveFrom = tileIndexInList(row, col)  # index of blank in the list
        stateCopy[moveFrom] = stateCopy[moveFrom + 3]  # swap blank with tile below
        stateCopy[moveFrom + 3] = 0  # place blank in tile below

    elif action[0] == 'left':  # get direction from action tuple
        moveFrom = tileIndexInList(row, col)  # index of blank in the list
        stateCopy[moveFrom] = state[moveFrom - 1]  # swap blank with left tile
        stateCopy[moveFrom - 1] = 0  # place blank in left tile

    elif action[0] == 'right':  # get direction from action tuple
        moveFrom = tileIndexInList(row, col)  # index of blank in the list
        stateCopy[moveFrom] = stateCopy[moveFrom + 1]  # swap blank with right tile
        stateCopy[moveFrom + 1] = 0  # place blank in right tile
    return stateCopy, 1  # return updated state and cost of step

## Iterative Deepening Search (IDS)

In [6]:
def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    global num_nodes  # use the global num_nodes var
    
    if state == goalState:
        return []
    if depthLimit == 0:
        return 'cutoff'

    cutoffOccurred = False

    for action in actionsF(state):  # generate the next possible game states for the puzzle
        num_nodes += 1  # increment number of nodes expanded
        childState, _ = takeActionF(state, action)  # underscore is syntax for annonymous variable that will never be used
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit - 1)
        if result == 'cutoff':
            cutoffOccurred = True
        elif result is not 'failure':
            result.insert(0, childState)
            return result

    if cutoffOccurred:
        return 'cutoff'
    else:
        return 'failure'

def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result is 'failure':
            return 'failure'
        if result is not 'cutoff':
            result.insert(0, startState)
            return result
    return 'cutoff'

### Checking for goal

In [7]:
def goalTestF_8p(state, goal):
    return state == goal

## Testing IDS on an 8-puzzle

8-puzzle with one start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

In [8]:
startState_8p = [1, 2, 3, 4, 0, 5, 6, 7, 8]

In [9]:
goalState1_8p = [1, 2, 3, 4, 0, 5, 6, 7, 8]
goalState2_8p = [1, 2, 3, 4, 5, 8, 6, 0, 7]
goalState3_8p = [1, 0, 3, 4, 5, 8, 2, 6, 7]

In [10]:
iterativeDeepeningSearch(startState_8p, goalState1_8p, actionsF_8p, takeActionF_8p, 10)

[[1, 2, 3, 4, 0, 5, 6, 7, 8]]

In [11]:
iterativeDeepeningSearch(startState_8p, goalState2_8p, actionsF_8p, takeActionF_8p, 20)

[[1, 2, 3, 4, 0, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 0, 6, 7, 8],
 [1, 2, 3, 4, 5, 8, 6, 7, 0],
 [1, 2, 3, 4, 5, 8, 6, 0, 7]]

In [12]:
iterativeDeepeningSearch(startState_8p, goalState3_8p, actionsF_8p, takeActionF_8p, 20)

[[1, 2, 3, 4, 0, 5, 6, 7, 8],
 [1, 0, 3, 4, 2, 5, 6, 7, 8],
 [0, 1, 3, 4, 2, 5, 6, 7, 8],
 [4, 1, 3, 0, 2, 5, 6, 7, 8],
 [4, 1, 3, 2, 0, 5, 6, 7, 8],
 [4, 1, 3, 2, 5, 0, 6, 7, 8],
 [4, 1, 3, 2, 5, 8, 6, 7, 0],
 [4, 1, 3, 2, 5, 8, 6, 0, 7],
 [4, 1, 3, 2, 5, 8, 0, 6, 7],
 [4, 1, 3, 0, 5, 8, 2, 6, 7],
 [0, 1, 3, 4, 5, 8, 2, 6, 7],
 [1, 0, 3, 4, 5, 8, 2, 6, 7]]

## Uninformed vs. Informed Search

IDS is great for uninformed searching, where we can't make inferences on what the next best move is. However, since the 8-puzzle is a game, we do have some ideas of what moves are better than others. With good enough heuristics (guesses), we could feed these and the current state of the game into the A\* search algorithm to make an informed search of the state space.

The benefits of using A\* is to cut down on the number of unecessary states we would search using an uniformed search such as IDS. Cutting down unecessary search paths is also known as **pruning**.

Below, I have implemented A\* search based off of [Prof. Chuck Anderson's](http://www.cs.colostate.edu/~anderson/cs440/doku.php) notes and [Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu).

## Finding Effective Branching Factor (ebf)

In [13]:
def ebf(nNodes, depth, precision=0.01):
    if nNodes == 1 and depth == 0:
        return 1
    if nNodes == 0:  # no nodes means no branching factor is possible
        return 0
    if nNodes == 1:  # branching factor can only be 1 with 1 node
        return ebf_helper(0, nNodes, nNodes, depth, precision)
    return ebf_helper(1, nNodes, nNodes, depth, precision)


def ebf_helper(lo, hi, nNodes, depth, precision=0.01):
    mid = (hi + lo) / 2  # aka branching factor b

    if mid == 1:
        nGuess = 1
        for d in range(1, depth):
            nNodes += mid ** d
    else:
        nGuess = (1 - (mid ** (depth + 1))) / (1 - mid)

    if abs(nGuess - nNodes) < precision:
        return mid

    if nGuess < nNodes:
        return ebf_helper(mid, hi, nNodes, depth, precision)
    else:  # nGuess > nNodes
        return ebf_helper(lo, mid, nNodes, depth, precision)

## A\* Search

In order to carry out A\* search, we need to represent our states as nodes with f, g, and h values where f is total cost to get to the goal, g is cost to the node so far, and h is an estimate of the remaining cost to reach the goal.

### Node Class

In [14]:
class Node:
    def __init__(self, state, f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

### A\* Implementation

In [15]:
def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))


def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fLimit):
    global num_nodes  # use the global num_nodes var
    
    if goalTestF(parentNode.state) is True:
        return [parentNode.state], parentNode.g     # return a list so upper recursion levels
                                                    # can construct solution path

    successors = []  # list of children to search

    for action in actionsF(parentNode.state):
        # Construct children nodes with childState, f, g, and h values
        childState, stepcost = takeActionF(parentNode.state, action)
        g = parentNode.g + stepcost
        h = hF(childState)
        f = max(g + h, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        successors.append(childNode)
        num_nodes += 1  # increment number of nodes expanded

    if not successors:  # if no children to search, then no solution
        return 'failure', float('inf')

    # Start of actual searching
    while True:                                 # Search through all the children
        successors.sort(key = lambda n: n.f)    # sort nodes by lowest f value
        bestSuccessor = successors[0]

        if bestSuccessor.f > fLimit:
            return 'failure', bestSuccessor.f

        alternativef = successors[1].f if len(successors) > 1 else float('inf')  # node with second-lowest f value

        # expand best child with best f value, reassign its f value to be returned value
        result, bestSuccessor.f = aStarSearchHelper(bestSuccessor, actionsF, takeActionF,
                                                    goalTestF, hF, min(fLimit, alternativef))

        # if result is not failure, it means we have found the solution and
        # can start backtracking to form the solution path
        if result is not 'failure':
            result.insert(0, parentNode.state)
            return result, bestSuccessor.f

Here is a simple example using graph search.

In [16]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [17]:
actions = actionsF_simple('a')
actions

[('b', 1), ('c', 1)]

In [18]:
takeActionF_simple('a', actions[0])

('b', 1)

In [19]:
goalTestF_simple('a', 'a')

True

In [20]:
h_simple('a', 'z')

1

In [21]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

Let's see what solution IDS gives us.

In [22]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

Hmm, so A\* gives us the same solution that IDS would have given us. So what is the benefit of using A\* over IDS? Well, let's try to do some behind the scenes analysis to figure this out.

## Effective Branching Factor (EBF)

In order to help us analyze these searches, we first need to explore a concept **known as effective branching factor**.

EBF is a measure of how many child nodes, on average, are generated for each parent node in a search. Given number of nodes expanded **n** and depth **d** at which solution was found, one can solve for the EBF **b** using the general formula:

$$
n = 1 + b + b^2 + \cdots + b^d
$$

or a faster way to calculate this:

$$
\frac{1-b^{d+1}}{1-b}
$$

A high branching factor and depth would translate to a large amount of nodes expanded, leading to more time being spent on searching. In our analysis of these searches, we will be finding the EBF and seeing how many nodes are expanded in each search. By the end of our analysis, we will be able to discern why A\* is better than IDS for searching when we can make informed choices.

In [23]:
def ebf(nNodes, depth, precision=0.01):
    if nNodes == 1 and depth == 0:
        return 1
    if nNodes == 0:  # no nodes means no branching factor is possible
        return 0
    if nNodes == 1:  # branching factor can only be 1 with 1 node
        return ebf_helper(0, nNodes, nNodes, depth, precision)
    return ebf_helper(1, nNodes, nNodes, depth, precision)


def ebf_helper(lo, hi, nNodes, depth, precision=0.01):
    mid = (hi + lo) / 2  # aka branching factor b

    if mid == 1:
        nGuess = 1
        for d in range(1, depth):
            nNodes += mid ** d
    else:
        nGuess = (1 - (mid ** (depth + 1))) / (1 - mid)

    if abs(nGuess - nNodes) < precision:
        return mid

    if nGuess < nNodes:
        return ebf_helper(mid, hi, nNodes, depth, precision)
    else:  # nGuess > nNodes
        return ebf_helper(lo, mid, nNodes, depth, precision)

First, some example output for the ebf function.

In [24]:
ebf(10, 3)

1.661376953125

The smallest argument values should be a depth of 0 and 1 node, as we must start from the root.

In [25]:
ebf(1, 0)

1

In [26]:
ebf(2, 1)

1.0078125

In [27]:
ebf(2, 1, precision=0.000001)

1.0000009536743164

In [28]:
ebf(200000, 5)

11.275596931956898

In [29]:
ebf(200000, 50)

1.2348192492705223

## Comparing A\* and IDS

I will organize the solution path depth, number of nodes generated, and the effective branching factor into a table using the `runExperiment(...)` below.

In [30]:
def runExperiment(goalState1, goalState2, goalState3, heuristics):
    global num_nodes
    num_nodes = 0   # zero out num_nodes to be safe
    print('{:>9}{:^35}{:^35}{:^35}'.format(' ', str(goalState1), str(goalState2), str(goalState3)))
    print('Algorithm' + '{:>10}{:>10}{:>10}'.format('Depth', 'Nodes', 'EBF') +
          '{:>15}{:>10}{:>10}'.format('Depth', 'Nodes', 'EBF') +
          '{:>15}{:>10}{:>10}'.format('Depth', 'Nodes', 'EBF'))
    print('{:<17}'.format('IDS'), end='')
    
    goals = [goalState1, goalState2, goalState3]
    
    for goalState in [goalState1, goalState2, goalState3]:
        ids = iterativeDeepeningSearch(startState_8p, goalState, actionsF_8p, takeActionF_8p, 20)
        depth = len(ids) - 1    # depth of IDS is just the length of the solution path - 1
        b = ebf(num_nodes, depth)
        print('{:}{:>10}{:>10.3f}{:>14}'.format(depth, num_nodes, b, ' '), end='')
        num_nodes = 0   # reset nodes expanded for next search
    print()

    print('{:<17}'.format('A*h1'), end='')
    for goal in goals:
        aStar, depth = aStarSearch(startState_8p, actionsF_8p, takeActionF_8p,
                                   lambda s: goalTestF_8p(s, goal),
                                   lambda s: heuristics[0](s, goal))
        b = ebf(num_nodes, depth)
        print('{:}{:>10}{:>10.3f}{:>14}'.format(depth, num_nodes, b, ' '), end='')
        num_nodes = 0  # reset nodes expanded for next search
    print()

    print('{:<17}'.format('A*h2'), end='')
    for goal in goals:
        # depth of A* is the stepcost returned by the search
        aStar, depth = aStarSearch(startState_8p, actionsF_8p, takeActionF_8p,
                                   lambda s: goalTestF_8p(s, goal),
                                   lambda s: heuristics[1](s, goal))
        b = ebf(num_nodes, depth)
        print('{:}{:>10}{:>10.3f}{:>14}'.format(depth, num_nodes, b, ' '), end='')
        num_nodes = 0  # reset nodes expanded for next search
    print()

    print('{:<17}'.format('A*h3'), end='')
    for goal in goals:
        aStar, depth = aStarSearch(startState_8p, actionsF_8p, takeActionF_8p,
                                   lambda s: goalTestF_8p(s, goal),
                                   lambda s: heuristics[2](s, goal))
        b = ebf(num_nodes, depth)
        print('{:}{:>10}{:>10.3f}{:>14}'.format(depth, num_nodes, b, ' '), end='')
        num_nodes = 0  # reset nodes expanded for next search
    print()

### Heuristic Functions

For `aStarSearch` I use the following three heuristic functions, two given by Prof. Chuck Anderson, and one I implemented from [Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu).

Given:
  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position

Heuristic I chose to implement:
  * `h3_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the cumulative Manhattan distance that the tiles are from their goal positions.

In [31]:
def h1_8p(state, goal):
    '''
    Returns a constant 0 as a heuristic
    i.e. basically does a breadth first search
    :param state: 
    :param goal: 
    :return 0: 
    '''
    return 0

def h2_8p(state, goal):
    '''
    Calculates the Manhattan Distance for the blank as a heuristic
    i.e. how far blank is away from its position in goal state
    :param state:
    :param goal:
    :return manhattan_dist:
    '''
    row, col = findTile_8p(state, 0)
    goalRow, goalCol = findTile_8p(goal, 0)

    return abs(row - goalRow) + abs(col - goalCol)  # manhattan distance

def h3_8p(state, goal):
    '''
    Calculates the Manhattan Distance for all the tiles as a heuristic
    i.e. how far, collectively, the tiles are away from their positions in the goal state
    :param state:
    :param goal:
    :return manhattan_dist:
    '''
    manhattan_dist = 0
    for tile in state:
        row, col = findTile_8p(state, tile)
        goalRow, goalCol = findTile_8p(goal, tile)
        manhattan_dist += abs(row - goalRow) + abs(col - goalCol)

    return manhattan_dist

In [32]:
runExperiment(goalState1_8p, goalState2_8p, goalState3_8p, [h1_8p, h2_8p, h3_8p])

             [1, 2, 3, 4, 0, 5, 6, 7, 8]        [1, 2, 3, 4, 5, 8, 6, 0, 7]        [1, 0, 3, 4, 5, 8, 2, 6, 7]    
Algorithm     Depth     Nodes       EBF          Depth     Nodes       EBF          Depth     Nodes       EBF
IDS              0         0     0.000              3        43     3.086              11    225850     2.954              
A*h1             0         0     0.000              3       116     4.488              11    643246     3.263              
A*h2             0         0     0.000              3        51     3.297              11    100046     2.733              
A*h3             0         0     0.000              3         9     1.578              11      3944     1.992              


### Discussing the results

From the table above one can see that A\* with the first heuristic expanded the most nodes out of all the searches. Why is this? Well, the reason lies in the fact that the heuristic returns a constant 0 as the guess for the remaining cost to get to the goal. In this case, A\* is, in practice, doing Breadth-First search, hence the high EBF. Since all the successors start with an f cost of 1, A\* will start exploring in a frontier-like fashion. When A\* expands a successor n, the f costs of the children of n will be more than the f cost of the nodes on the previous levels. It won't be until A\* has explored all the nodes on one level will it then proceed to the next. The heuristic here is essentially useless as it does not inform A\* any useful information.

A\* with the second and third heuristics expanded significantly less nodes than IDS. Let's first look at A\* with the second heuristic versus IDS. The second heuristic was the Manhattan distance of the blank to its position in the goal state. This is a useful heuristic because A\* will choose to make moves that are the lowest distance from the goal (i.e. get it to the solution the quickest).

A\* with the third heuristic is the one I chose to implement: the Manhattan distance of all the tiles to their positions in the goal state. This heuristic is admissible because it never overestimates the true cost to get to the goal. It never overestimates the true cost because the shortest path for any tile to its position in the goal state is the Manhattan distance. Therefore, the sum of all Manhattan distances of all tiles to their positions in the goal state will always be less than or equal to the true cost to get to the goal state.

I think the third heuristic reduced the number of nodes expanded by a significant amount because instead of just the Manhattan distance of the blank, it was able to pool together the Manhattan distances of all the tiles to their correct positions. This collective Manhattan distance gives A\* a more informed decision, and therefore it expands significantly less nodes than any of the other searches.